# Functions

- Two functions are same type, if the sequence of parameter types and sequence of return types are same
- Go has no concept of default parameter values, and also no way to specify arguments by name.

## Function Values

- Functions can be passed around as:
    ```go
    func compute(fn func(float64, float64) float64) float64 {
        return fn(3, 4)
    }

    func main() {
        hypot := func(x, y float64) float64 {
            return math.Sqrt(x*x + y*y)
        }
        fmt.Println(hypot(5, 12))
    }
    ```

## Anonymous Functions

- Use the function literal to denote a function value within any expression
    ```go
    func createAdder(a int) func(int) int {
        return func(n int) int {
            return n + a 
        }   
    }
    func main() {
        addbytwo := createAdder(2)
        fmt.Println(addbytwo(4))    // 6
        fmt.Println(addbytwo(3))    // 5
    }
    ```

- Creating adder

    ```go
    func clo() func() int {
        var a int 
        return func() int {
            a++
            return a
        }   
    }
    func main() {
        adder := clo()
        fmt.Println(adder())  // 1
        fmt.Println(adder())  // 2
        fmt.Println(adder())  // 3
    }
    ```

- Function values like these are called closures.
- Above you must have seen that the lifetime of variable is not defined by its scope.
- With anonymous function using recursion you should declare it first.

## Function Closure

- Go functions may be closures. A closure is a function value that references variables from outside its body. The function may access and assign to the referenced variables; in this sense the function is "bound" to the variables. For example, the adder function returns a closure. Each closure is bound to its own sum variable.

    ```go
    func adder() func(int) int {
        sum := 0
        return func(x int) int {
            sum += x
            return sum
        }
    }

    func main() {
        pos, neg := adder(), adder()
        for i := 0; i < 10; i++ {
            fmt.Println(
                pos(i),
                neg(-2*i),
            )
        }
    }
    ```
    output
    ```bash
    0 0
    1 -2
    3 -6
    6 -12
    10 -20
    15 -30
    21 -42
    28 -56
    36 -72
    45 -90
    ```

## Variadic Functions

- Functions with varying number of arguments

    ```go
    func mult(nums ...int) int {
        product := 1
        for _, n := range nums {
            product *= n
        }   
        return product
    }
    fmt.Println(mult(5, 7, 9)) // 315
    ```

- `...int` above behaves like slice within the function body.
- Internally caller makes an array of the values and copies the argument into it, and passes a slice of the entire array to the function.
- So we can also do the following as well:

    ```go
    val := []int{1, 2, 3, 4}
    fmt.Println(mult(val...))    // 24
    ```
- Passing values of the slice using ellipsis after the final argument.

## `defer` keyword

- Defer: A defer statement defers the execution of a function until the surrounding function returns. 
    ```go
    defer fmt.Println("world")
    fmt.Println("hello", i)
    ```
    
    Above will print 'hello' and then 'world'
    
- `defer` executes statements when function ends, this is like a `finally` block to `try`, `catch` in other languages.
- So if the function exits normally with return or exits with error or panics, defer is gonna be executed. 
- Execution sequence is opposite to the sequence of calling defer.

- The best place to use defer is with the pair of statements like open/close, lock/unlock. So this helps to release the resources held. And the `defer` statements which release resources should be written immediately after the acquiring statment.

## Managing Errors

- **Propogate errors**: Error in called function is sent back into this function and this function again sends on the error. So in the process of passing on errors, we can add information to error, using, `fmt.Errorf`. So basically chained errors will give more information.

- **Retry failed action**: Actions that has any form of IO operation is subject to time-based failure, so in that case we can use such a strategy, try again, with some time-outs or something.

- Normally all functions with significant amount of work do return errors.

- If error is ignored, document it why it is done so.

- Error handling in golang has a widely followed process, after a function call that returns error, the failure condition is checked first with `if err != nil` and if such a condition is found we do return with some error handling message, while success condition does not go into else block of check, but subsequent block of code is considered as success condition.

- Normally functions in go, they check for all the failure causing conditions and the last one is success!

## `panic`

- `panic` is for runtime errors
- When some error occurs at runtime the program panics
- You can also use `panic` function in your code on a check of a condition that is impossible logically.
- Even if go's panic may seem similar to exceptions in other language, but it is not.
- Exception check in other language is similar to error check after function call in go.
- Panic should not be used for "expcted" errors, like, failing IO, incorrect input, misconfiguration, etc. those kinda errors should be handled gracefully.
- Panic causes program to crash so should be used attentively.

## `log` package

- `log.Fatalf` is equivalent to `Printf()` followed by a call to `os.Exit(1)`.

- Below two methods can be used to change the behavior of logs, and can be set anywhere and from then on its set.
    - `log.SetPrefix` sets the output prefix for the standard logger.
    - `log.SetFlags` sets the output flags for the standard logger. If the value is `0` then it willnot show date and time.